In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.7MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import permutation_importance

from ast import literal_eval
from tqdm import tqdm_notebook

In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix'

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df = pd.read_csv('data/Mens_Shoe_Prices.csv', low_memory=False)

In [0]:
df_usd = df[df.prices_currency == 'USD'].copy()

In [0]:
df_usd['prices_amountmin'] = df_usd.prices_amountmin.astype(np.float)

In [0]:
filter_max = np.percentile(df_usd.prices_amountmin, 99)

In [0]:
df = df_usd[df_usd['prices_amountmin'] < filter_max]

In [15]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
df['manufacturer_cat'] = df['manufacturer'].factorize()[0]
df['colors_cat'] = df['colors'].factorize()[0]
df['name_cat'] = df['name'].factorize()[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [0]:
feats = ['brand_cat', 'manufacturer_cat', 'colors_cat', 'name_cat']

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=15, random_state=2020)):
  X = df[feats].values
  y = df['prices_amountmin'].values
  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [12]:
run_model(feats)

(-58.96236972061157, 5.091024678885326)

In [0]:
model = RandomForestRegressor (max_depth=10, n_estimators=100, random_state=2020)

In [14]:
run_model(feats, model)

(-54.515545068929136, 3.1294377632381685)

In [22]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  features = literal_eval(x.replace('\\"', '"'))

  for feat in features:
    key = feat['key'].lower().strip()
    value = feat['value'][0].lower().strip()

    output_dict[key] = value
  return output_dict

df['features_passed'] = df['features'].map(parse_features)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [26]:
keys = set()
df['features_passed'].map(lambda x: keys.update(x.keys()))

0        None
1        None
2        None
3        None
4        None
         ... 
19382    None
19383    None
19384    None
19385    None
19386    None
Name: features_passed, Length: 18280, dtype: object

In [27]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_passed.map(lambda feats: feats[key] if key in feats else np.nan)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [28]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_bed size', 'feat_hat style', 'feat_clasp type', 'feat_use',
       'feat_polarised', 'feat_sports league', 'feat_product name',
       'feat_temple length', 'feat_lens technology', 'feat_leg'],
      dtype='object', length=529)